<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [78]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [79]:
try:
    df = pd.read_csv('/datasets/insurance.csv')
except:
    print('При чтение файла возникал ошибка')

In [80]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Не изменится.

**Обоснование (версия 1):** $w' = ((XP)^T (XP))^{-1} (XP)^T y = (P^TX^TXP)^{-1}P^TX^Ty = P^{-1}X^{-1}(X^T)^{-1}(P^T)^{-1}P^TX^Ty = P^{-1}(X^TX)^{-1}X^Ty=P^{-1}w$

**Обоснование (версия 2):** $w' = ((XP)^T (XP))^{-1} (XP)^T y = (P^TX^TXP)^{-1}P^TX^Ty = (X^TXP)^{-1}(P^T)^{-1}P^TX^Ty = P^{-1}(X^TX)^{-1}X^Ty=P^{-1}w$

## Алгоритм преобразования

**Алгоритм**

1. Генерируем случайную обратимую квадратную матрицу P размера МхМ, где М - количество признаков в исходной матрице признаков 
2. Умножаем матрицу признаков на матрицу P, получаем зашифрованную матрицу признаков
3. Обучаем линейную регрессию на зашифрованной матрице признаков
4. Предсказания, полученные за зашифрованных данных, будут соответствовать предсказаниям, полученным на незашифрованных данных.

**Обоснование**

При умножении признаков на обратимую матрицу, меняются только значения признаков,но не их взаимосвязь. Коэффициенты линейной регрессии определяются взаимосвязями между признаками. Поэтому параметры линейной регрессии на исходных данных и на преобразованных будут связаны линейным преобразованием, и качество модели не изменится.

При использовании необратимой матрицы, некоторые признаки могут быть линейно выражены через другие, что приведет к потере информации и снижению качества линейной регрессии.

## Проверка алгоритма

Рассчитаем метрику R2 на исходных данных

In [81]:
features_train = df.drop(['Страховые выплаты'], axis=1)
target_train = df['Страховые выплаты']

In [82]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345)

In [83]:
model = LinearRegression()

In [84]:
model.fit(features_train,target_train)

LinearRegression()

In [85]:
prediction = model.predict(features_valid)

In [86]:
r2 = r2_score(target_valid, prediction)
print(f"R2 score: {r2:.5f}")

R2 score: 0.43523


Теперь расчитаем метрику R2 на данных умноженных на необратимую матрицу размерности равной количеству признаков.

In [87]:
d = features_train.shape[1]

In [88]:
P = np.random.rand(d, d)
while np.linalg.det(P) == 0:
    P = np.random.rand(d, d)

In [89]:
P_inv = np.linalg.inv(P)

In [90]:
features_train_protected = features_train.dot(P)

In [91]:
model_protected = LinearRegression().fit(features_train_protected, target_train)

In [92]:
features_valid_protected = features_valid.dot(P)

In [93]:
prediction_protected = model.predict(features_valid_protected)

In [94]:
r2_protected = r2_score(target_valid, prediction_protected)
print(f"R2 score (protected): {r2:.5f}")

R2 score (protected): 0.43523


**Вывод**<br>
Значение метрики R2 на исходных данных и на зашифрованных - совпали.